# Camille Girard Think Bayes Homework 5

This notebook presents example code and exercise solutions for Think Bayes.

Camille Girard

MIT License: https://opensource.org/licenses/MIT

In [2]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import classes from thinkbayes2
from thinkbayes2 import Pmf, Suite, Beta, MakeGammaPmf, MakeJoint, Joint
import thinkbayes2

import matplotlib.pyplot as plt
import thinkplot
import pandas as pd
import numpy as np
from scipy.stats import norm, hypergeom
import math
from thinkbayes2 import MakePoissonPmf, Cdf, Pmf, MakeGammaPmf
from scipy.special import expit, logit
from itertools import product
import pymc3 as pm

class BayesTable(pd.DataFrame):
    def __init__(self, hypo, prior=1):
        columns = ['hypo', 'prior', 'likelihood', 'unnorm', 'posterior']
        super().__init__(columns=columns)
        self.hypo = hypo
        self.prior = prior
    
    def mult(self):
        self.unnorm = self.prior * self.likelihood
        
    def norm(self):
        nc = np.sum(self.unnorm)
        self.posterior = self.unnorm / nc
        return nc
    
    def update(self):
        self.mult()
        return self.norm()
    
    def reset(self):
        return BayesTable(self.hypo, self.posterior)

ERROR:root:The 'ast_node_interactivity' trait of a ZMQInteractiveShell instance must be any of ['all', 'last', 'last_expr', 'none'], but a value of 'last_expr_or_assign' <class 'str'> was specified.
/home/cgirard/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Climate Change

Climate Change

In [3]:
!wget https://www.ncdc.noaa.gov/cag/global/time-series/0.0,0.0/land_ocean/ytd/12/1880-2018.csv

--2018-11-11 15:00:25--  https://www.ncdc.noaa.gov/cag/global/time-series/0.0,0.0/land_ocean/ytd/12/1880-2018.csv
Resolving www.ncdc.noaa.gov (www.ncdc.noaa.gov)... 205.167.25.172, 205.167.25.171, 2610:20:8040:2::172, ...
Connecting to www.ncdc.noaa.gov (www.ncdc.noaa.gov)|205.167.25.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133 [text/plain]
Saving to: ‘1880-2018.csv.2’

1880-2018.csv.2     100%[===================>]     133  --.-KB/s    in 0s      

2018-11-11 15:00:35 (3.14 MB/s) - ‘1880-2018.csv.2’ saved [133/133]



In [11]:
columns = ['Year', 'Value']
df = pd.read_csv('1880-2018.csv', parse_dates=[0])
df.drop(labels=[3, 24], inplace=True)
df.head

<bound method NDFrame.head of                   0.0 deg N  0.0 deg E Temperature Anomalies  \
0    Units: Degrees Celsius                              NaN   
1    Base Period: 1981-2010                              NaN   
2            Missing: -9999                              NaN   
4                      1896                            -1.05   
5                      1902                            -1.50   
6                      1903                            -1.94   
7                      1904                            -2.10   
8                      1905                            -1.74   
9                      1906                            -1.68   
10                     1907                            -1.15   
11                     1908                            -1.53   
12                     1909                            -0.78   
13                     1910                            -0.54   
14                     1911                            -1.68   
15        

In [12]:
loc = "/home/cgirard/ThinkBayes2/examples/" # Camille's laptop
document = loc + "temp_and_co2.csv"
df = pd.read_csv(document)
df.head()

FileNotFoundError: File b'/home/cgirard/ThinkBayes2/examples/temp_and_co2.csv' does not exist